<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Table-Valued Python UDFs to demonstrate JSON Expansion</h1>
    </div>
</div>

### Setup: Database Schema

In [1]:
%%sql
-- Clean up any existing resources
DROP TABLE IF EXISTS orders;

-- Create sample table with nested JSON
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT NOT NULL,
    order_date DATE NOT NULL,
    line_items JSON NOT NULL,
    metadata JSON
);

++
||
++
++

### Insert Sample Data with Nested JSON

In [2]:
%%sql
INSERT INTO orders (order_id, customer_id, order_date, line_items, metadata) VALUES
(1001, 501, '2026-01-15',
 '[{"product_id": "P123", "product_name": "Laptop", "quantity": 1, "unit_price": 1299.99},
   {"product_id": "P456", "product_name": "Mouse", "quantity": 2, "unit_price": 29.99}]',
 '{"source": "web", "promo_code": "WINTER25", "shipping_method": "express"}'),

(1002, 502, '2026-01-16',
 '[{"product_id": "P789", "product_name": "Keyboard", "quantity": 1, "unit_price": 149.99},
   {"product_id": "P101", "product_name": "Monitor", "quantity": 2, "unit_price": 399.99},
   {"product_id": "P202", "product_name": "USB Cable", "quantity": 3, "unit_price": 12.99}]',
 '{"source": "mobile_app", "shipping_method": "standard"}'),

(1003, 503, '2026-01-17',
 '[{"product_id": "P303", "product_name": "Headphones", "quantity": 1, "unit_price": 89.99}]',
 '{"source": "web", "promo_code": "SAVE10"}');

3 rows affected.

++
||
++
++

### Verify Sample Data

In [3]:
%%sql
-- Review the raw JSON structure
SELECT
    order_id,
    customer_id,
    line_items,
    metadata
FROM orders
LIMIT 3;

3 rows affected.

order_id,customer_id,line_items,metadata
1002,502,"[{'product_id': 'P789', 'product_name': 'Keyboard', 'quantity': 1, 'unit_price': 149.99}, {'product_id': 'P101', 'product_name': 'Monitor', 'quantity': 2, 'unit_price': 399.99}, {'product_id': 'P202', 'product_name': 'USB Cable', 'quantity': 3, 'unit_price': 12.99}]","{'shipping_method': 'standard', 'source': 'mobile_app'}"
1001,501,"[{'product_id': 'P123', 'product_name': 'Laptop', 'quantity': 1, 'unit_price': 1299.99}, {'product_id': 'P456', 'product_name': 'Mouse', 'quantity': 2, 'unit_price': 29.99}]","{'promo_code': 'WINTER25', 'shipping_method': 'express', 'source': 'web'}"
1003,503,"[{'product_id': 'P303', 'product_name': 'Headphones', 'quantity': 1, 'unit_price': 89.99}]","{'promo_code': 'SAVE10', 'source': 'web'}"


### Define Table-Valued Python UDF: Line Items Expansion

In [4]:
import typing
import json
import pandas as pd
import logging
from singlestoredb.functions import udf, Table

# Configure logging
logger = logging.getLogger(__name__)

class LineItemOutput(typing.NamedTuple):
    product_id: str
    product_name: str
    quantity: int
    unit_price: float
    line_total: float

@udf(returns=LineItemOutput)
async def expand_line_items(json_array: str) -> Table[pd.DataFrame]:
    """
    Expands a JSON array of line items into individual rows.

    Parameters
    ----------
    json_array : str
        JSON string containing array of line item objects

    Returns
    -------
    Table[pd.DataFrame]
        Table with columns: product_id, product_name, quantity,
        unit_price, line_total (computed)
    """
    # Handle null/empty input gracefully
    if not json_array:
        logger.debug("Received null or empty json_array, returning empty result")
        return Table(pd.DataFrame(columns=['product_id', 'product_name',
                                           'quantity', 'unit_price', 'line_total']))

    # Parse JSON - let decode errors bubble up to caller
    items = json.loads(json_array)

    if not items:
        logger.debug("Parsed empty array, returning empty result")
        return Table(pd.DataFrame(columns=['product_id', 'product_name',
                                           'quantity', 'unit_price', 'line_total']))

    rows = []
    for idx, item in enumerate(items):
        try:
            quantity = int(item.get('quantity', 0))
            unit_price = float(item.get('unit_price', 0.0))

            rows.append({
                'product_id': item.get('product_id', ''),
                'product_name': item.get('product_name', ''),
                'quantity': quantity,
                'unit_price': unit_price,
                'line_total': quantity * unit_price
            })
        except (ValueError, TypeError) as e:
            # Log the problematic item but continue processing others
            logger.warning(f"Skipping item at index {idx} due to type conversion error: {e}")
            continue

    return Table(pd.DataFrame(rows))

### Define Table-Valued Python UDF: Metadata Expansion

In [5]:
class MetadataOutput(typing.NamedTuple):
    key: str
    value: str

@udf(returns=MetadataOutput)
async def expand_metadata(json_obj: str) -> Table[pd.DataFrame]:
    """
    Expands a JSON object into key-value pairs (one row per field).
    Useful for dynamic schemas where field names aren't known in advance.

    Parameters
    ----------
    json_obj : str
        JSON string containing object with arbitrary fields

    Returns
    -------
    Table[pd.DataFrame]
        Table with columns: key, value
    """
    # Handle null/empty input gracefully
    if not json_obj:
        logger.debug("Received null or empty json_obj, returning empty result")
        return Table(pd.DataFrame(columns=['key', 'value']))

    # Parse JSON - let decode errors bubble up to caller
    obj = json.loads(json_obj)

    rows = [{'key': k, 'value': str(v)} for k, v in obj.items()]

    if not rows:
        logger.debug("Parsed empty object, returning empty result")
        return Table(pd.DataFrame(columns=['key', 'value']))

    return Table(pd.DataFrame(rows))

## Start the Python UDF Server
This registers all @udf annotated functions as external functions in your database.

In [6]:
import singlestoredb.apps as apps
import logging

# Configure logging level
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

connection_info = await apps.run_udf_app()
logger.info("Python UDF server is running")
logger.info(f"Functions registered with '_test' suffix for notebook testing")

Python UDF registered at https://apps.us-east-1.cloud.singlestore.com/pythonudfs/a893f1c3-7196-44f7-9ff7-7408e8e03ae3/interactive/
✓ Python UDF server is running
✓ Functions registered with '_test' suffix for testing


## Run the following commands in a new Editor for demonstrating production usecase

#### Verify Function Registration

In [7]:
%%sql
-- Check registered functions
SHOW FUNCTIONS LIKE '%expand%';

4 rows affected.

Functions_in_temp (%expand%),Function Type,Definer,Data Format,Runtime Type,Link,Options
expand_line_items,External Table Valued Function,5b45981c-8b09-4ead-bec0-09547d7b9a97@%,ROWDAT_1,Managed Service,,
expand_line_items_test,External Table Valued Function,5b45981c-8b09-4ead-bec0-09547d7b9a97@%,ROWDAT_1,Managed Service,,
expand_metadata,External Table Valued Function,5b45981c-8b09-4ead-bec0-09547d7b9a97@%,ROWDAT_1,Managed Service,,
expand_metadata_test,External Table Valued Function,5b45981c-8b09-4ead-bec0-09547d7b9a97@%,ROWDAT_1,Managed Service,,


In [8]:
%%sql
-- View function signature
SHOW CREATE FUNCTION expand_line_items_test;

1 rows affected.

Function,sql_mode,Create Function,character_set_client,collation_connection
expand_line_items_test,"STRICT_ALL_TABLES,NO_AUTO_CREATE_USER","CREATE OR REPLACE EXTERNAL FUNCTION `expand_line_items_test`(json_array text CHARACTER SET utf8mb4 COLLATE utf8mb4_bin NOT NULL) RETURNS TABLE(`product_id` text CHARACTER SET utf8mb4 COLLATE utf8mb4_bin NOT NULL, `product_name` text CHARACTER SET utf8mb4 COLLATE utf8mb4_bin NOT NULL, `quantity` bigint(20) NOT NULL, `unit_price` double NOT NULL, `line_total` double NOT NULL) DEFINER = '5b45981c-8b09-4ead-bec0-09547d7b9a97'@'%' AS MANAGED SERVICE 'https://apps.us-east-1.cloud.singlestore.com/pythonudfs/a893f1c3-7196-44f7-9ff7-7408e8e03ae3/interactive/' FORMAT ROWDAT_1;",utf8,utf8mb4_bin


#### Why Not Use Native JSON Functions?

Native JSON functions work for simple extractions but become verbose and brittle for complex transformations.

```sql
%%sql
-- Native approach: requires knowing array size, returns strings, no computed fields
SELECT
    order_id,
    JSON_EXTRACT_STRING(JSON_ARRAY_ELEMENT(line_items, 0), 'product_name') as item1_name,
    JSON_EXTRACT_STRING(JSON_ARRAY_ELEMENT(line_items, 0), 'quantity') as item1_qty,
    JSON_EXTRACT_STRING(JSON_ARRAY_ELEMENT(line_items, 1), 'product_name') as item2_name
    -- Breaks if order has different number of items!
FROM orders
LIMIT 3;
```

**With Table-Valued Python UDFs:**
- Dynamic: Works regardless of array size
- Clean: Single function call vs. nested extractions
- Computed fields: Calculate line_total during expansion
- Type safety: Returns proper int/float types
- Reusable: Define once, use everywhere

Publish the above code as a Python UDF and then run the following SQL code to test its working.

### Use Case 1: Basic JSON Array Expansion

Instead of querying raw JSON, we've normalized the nested array into relational rows that are easy to filter and aggregate.
```sql
-- Flatten nested line items into individual rows
SELECT
    o.order_id,
    o.customer_id,
    o.order_date,
    li.product_id,
    li.product_name,
    li.quantity,
    li.unit_price,
    li.line_total
FROM orders o
CROSS JOIN expand_line_items(o.line_items) AS li
ORDER BY o.order_id, li.product_id;

```


### Use Case 2: Product Revenue Analysis

We can aggregate across all orders to understand product performance
```sql
-- Calculate total revenue and units sold per product
SELECT
    li.product_name,
    SUM(li.quantity) as total_units_sold,
    SUM(li.line_total) as total_revenue,
    COUNT(DISTINCT o.order_id) as num_orders,
    ROUND(AVG(li.unit_price), 2) as avg_unit_price
FROM orders o
CROSS JOIN expand_line_items(o.line_items) AS li
GROUP BY li.product_name
ORDER BY total_revenue DESC;

```

### Use Case 3: Dynamic Metadata Extraction

We don't need to know metadata field names in advance. The TVF extracts whatever keys exist
```sql
-- Find all orders that used promo codes (filtering on dynamically extracted fields)
SELECT
    o.order_id,
    o.customer_id,
    o.order_date,
    m.value as promo_code_used
FROM orders o
CROSS JOIN expand_metadata(o.metadata) AS m
WHERE m.key = 'promo_code'
ORDER BY o.order_id;

```



### Use Case 4: Multi-TVF Analytics - Revenue by Channel

Join multiple TVF expansions to answer complex business questions: Which marketing channels drive the most revenue?"

```sql
-- Calculate revenue breakdown by traffic source (combines both TVFs)
SELECT
    m.value as channel,
    COUNT(DISTINCT o.order_id) as orders,
    SUM(li.line_total) as total_revenue,
    ROUND(AVG(li.line_total), 2) as avg_line_item_value,
    SUM(li.quantity) as total_units
FROM orders o
CROSS JOIN expand_line_items(o.line_items) AS li
CROSS JOIN expand_metadata(o.metadata) AS m
WHERE m.key = 'source'
GROUP BY m.value
ORDER BY total_revenue DESC;

```


### Use Case 5: Full Order Denormalization with Pivot

Complete denormalization - transform nested JSON into a flat, analytics-ready table. This usabe by BI tools, dashboards, and reporting, and reporting pipelines.

```sql
-- Create fully denormalized view combining line items with pivoted metadata
SELECT
    o.order_id,
    o.customer_id,
    o.order_date,
    li.product_name,
    li.quantity,
    li.line_total,
    MAX(CASE WHEN m.key = 'source' THEN m.value END) as order_source,
    MAX(CASE WHEN m.key = 'promo_code' THEN m.value END) as promo_code,
    MAX(CASE WHEN m.key = 'shipping_method' THEN m.value END) as shipping_method
FROM orders o
CROSS JOIN expand_line_items(o.line_items) AS li
CROSS JOIN expand_metadata(o.metadata) AS m
GROUP BY o.order_id, o.customer_id, o.order_date, li.product_name, li.quantity, li.line_total
ORDER BY o.order_id;

```

### When to Use Table-Valued Python UDFs

### Best Use Cases
✓ **Complex transformations**: Logic difficult with native SQL/JSON functions
✓ **Dynamic schemas**: Field names/structure varies by record
✓ **Computed fields**: Calculations during expansion (like line_total)
✓ **Reusable logic**: Define once, use across queries and applications

### Optimization Tips
1. Materialize frequent queries in views
2. Apply WHERE clauses before TVF when possible
3. Index supporting columns (order_id, customer_id, etc.)
4. Use EXPLAIN to verify query plans

### When NOT to Use
✗ Simple single-field extractions (use JSON_EXTRACT_STRING)
✗ Very high-frequency point queries (consider denormalization)

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>